<a href="https://colab.research.google.com/github/Jun-629/20MA573/blob/master/src/Hw9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider 2-d PDE

\begin{align}
\frac{1}{2}\Delta v(x) - v(x) + x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2} = 0, x \in O = (0,1)^2 \tag{1} \\
\end{align}

with its boundary data
$$v(x) = (x_1 - \frac{1}{2})^2 + (x_2 - \frac{1}{2})^2, x \notin O.$$
- Show that exact solution is
$$v(x) = (x_1 - \frac{1}{2})^2 + (x_2 - \frac{1}{2})^2.$$

__Pf:__

\begin{aligned}
L.H.S &= \frac{1}{2} \sum_{i=1}^2 \partial_{ii} v(x) - v(x) + x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2} \\
&= \frac{1}{2} \cdot (2+2) - (x_1^2 - x_1 + \frac{1}{4}) - (x_2^2 - x_2 + \frac{1}{4}) + x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2} \\
&= 1 - x_1^2 - x_2^2 + x_1 + x_2 - \frac{1}{2} + x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2} \\
&= R.H.S \\
\end{aligned}

Also, $v(x)$ meets the boundary data, i.e., for $x \notin O$.

Therefore, the exact solution is 
$$v(x) = (x_1 - \frac{1}{2})^2 + (x_2 - \frac{1}{2})^2.$$

- Identify $\gamma, \ell^h, p^h$ in the CFD solution given by

\begin{align}
v(x) = \gamma \{ \ell^h(x) + \sum_{i=1}^d p^h(x+he_i|x)v(x+he_i) + p^h(x-he_i|x)v(x-he_i) \}. \tag{2} \\
\end{align}

__Soln:__

For CFD Solution, we already have that:

\begin{cases}
\partial_i v(x) = \frac{v(x+he_i) - v(x-he_i)}{2h} \\
\partial_{ii} v(x) = \frac{v(x+he_i) - 2v(x) + v(x-he_i)}{h^2} \\
\end{cases}

Then, denote $v = v(x)$, $v_i^+ = v(x+he_i)$ and $v_i^- = v(x-he_i)$. Take these equations into the equation $(1)$, we will have that:
$$\frac{1}{2} \sum_{i=1}^2 \frac{v_i^+ - 2v + v_i^-}{h^2} - v + x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2} = 0 $$

\begin{aligned}
(\frac{2}{h^2} + 1)v &= \sum_{i=1}^2 \frac{1}{2h^2} v_i^+ + \sum_{i=1}^2 \frac{1}{2h^2} v_i^- + x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2} \\
v(x) = v &= \frac{2}{2+h^2} \cdot \{ (x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2}) + \sum_{i=1}^2 \frac{1}{4} v_i^+ + \sum_{i=1}^2 \frac{1}{4} v_i^- \} \\
\end{aligned}

Thus, to compare the above equation to the equation $(2)$, we can deduce that

\begin{cases}
\gamma = \frac{2}{2+h^2}, \\
\ell^h(x) = x_1^2 + x_2^2 - x_1 - x_2 - \frac{3}{2}, \\
d = 2, \\
p^h(x \pm he_i|x) = \frac{1}{4}. \\
\end{cases}




- For $\{ h = 2^{-i}, i = 2,3,4,5 \}$, compute CFD solution and find maxnorm of error.

In [0]:
import numpy as np

def VI(hat_eps, N):       # N: max iteration, hat_eps: tolerance
  flag = 1
  n = 0
